## Connect to ICW:

In [1]:
import cadspy

In [2]:
username = str(input('User number'))
icw = cadspy.DatabaseConnection(system='ICW', user=username)

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:salmon;font-family:Verdana,sans-serif;font-size:16px;">

<font size="4">Can't connect to ICW? Instructions on how to get access to it in the links below:

</font>
</div>

- Step 1: [Requesting Access to GitHub](https://baplc.sharepoint.com/sites/ask/SitePages/Requesting-Access-to-GitHub.aspx)

Once your access to GitHub has been aproved, you need to:

- Step 2: [Request access to British-Ent GitHub organisation](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/Corporate_Directory_git.md)

And

- Step 3: [Setup Sagemaker Studio to access GitHub](https://github.com/BritishAirways-Ent/insight-processes/blob/main/onboarding/sagemaker_to_github_setup.md)

<br>

Below are some packages to get you started. You don't have to use them but you may find them useful!

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
# diplay all rows and cols when using 'dataframe'.head() or 'dataframe'.tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<br>

### Data

#### S19 Lounge Eligibility data

In [5]:
query = """

sel * from LDB_SBOX_OR.HACKATHON_OPS_LOUNGE_ELIGIBILITY

"""

df_lounge_eligibility = icw.queryToDataframe(query)

In [6]:
df_lounge_eligibility.head(2)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13


In [7]:
df_lounge_eligibility.shape

(1534750, 12)

In [8]:
# a look to a particular flight number and date

#mask = (df_lounge_eligibility['DISCHARGE_STN_CD'] == 'GCM   ') & (df_lounge_eligibility['GMT_UPLIFT_DT'] == dt.date(2019,9,12) )

#df_lounge_eligibility[mask]

#### S19 Flight info

In [9]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_FLIGHT_INFO

"""

df_flight_info = icw.queryToDataframe(query)

In [10]:
df_flight_info.head(2)
df_flight_info.dtypes

GMT_PLND_DEP_TS    datetime64[ns]
GMT_ACT_DEP_TS     datetime64[ns]
OPG_ALN_CD                 object
OPG_FLT_NO                  int64
ACT_DEP_STN_CD             object
ACT_DEP_TML_CD             object
PLND_ARR_STN_CD            object
ACT_ARR_STN_CD             object
IATA_AC_TYP_CD             object
ACT_AC_TYP_CD              object
ROUTE                      object
dtype: object

#### Station Code Decode

In [11]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_COUNTRY_DECODE

"""

df_country = icw.queryToDataframe(query)

In [12]:
df_country.head(2)

,ROUTE,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,LHRINV,GB,United Kingdom and Northern Ireland,UK,UK
1,LHRSVO,RU,Russia in Europe,EASTERN EUROPE,EUROPE EXC UK


#### Additional Station Decodes from ICW reference table

In [13]:
# There are lots of missing destinations which are in lounge_elig but not in df_country.
# They are all new (since ~2019) arrivals to Heathrow.
# They are not in sandbox country dataset, but they are in a reference table on ICW.
# Load that ICW reference table and filter for results not in our merged table, but that are in lounge_elig.
# This should result in zero nan values.
query = """
SELECT STN_CD, COUNTRY_CD, COUNTRY_NM, CORP_GEOG_CTRY_GRP_NM, CORP_GEOG_CONTINENT_NM
FROM REF_GEOG_LOC_HIERARCHY
"""
df_additional_country_decodes=icw.queryToDataframe(query)
df_additional_country_decodes.shape # we will remove destinations already in df_country and not in df_lounge_elig later.

(10975, 5)

In [14]:
df_additional_country_decodes.head(5)

,STN_CD,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM
0,RFS,NI,Nicaragua,CENTRAL AMERICA,SOUTH AMERICA INC CARIBBEAN
1,COO,BJ,Benin,WEST AFRICA,AFRICA
2,SFY,US,United States,USA,NORTH AMERICA
3,HGO,CI,Cote d'Ivoire,WEST AFRICA,AFRICA
4,RKA,PF,French Polynesia,PACIFIC ISLANDS,AUSTRALASIA PACIFIC


#### Aircraft Type

In [15]:
query = """

select * from LDB_SBOX_OR.HACKATHON_OPS_AC_TYPE

"""

df_acft_typ = icw.queryToDataframe(query)

In [16]:
df_acft_typ.head(2)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235


In [17]:
df_acft_typ.shape

(35, 7)

In [18]:
df_acft_typ

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,320,A3,NB,0,24,0,132
1,332,W9,WB,0,30,21,235
2,339,W9,WB,0,30,21,234
3,777,K7,WB,0,48,24,203
4,781,X8,WB,7,49,35,165
5,351,K5,WB,0,56,56,219
6,321,M6,NB,0,23,0,131
7,32Q,N6,NB,0,32,0,172
8,789,L8,WB,8,42,39,127
9,32A,H3,NB,0,20,0,150


<br>

### Pre-processing

*Hint:* It is always worth checking the format of each of the columns in your dataframes before trying to do any work with them. To do so, you can make use of the `headers_and_first_row` function below.


In [19]:
def headers_and_first_row(df):
    '''
    print headers and first row of a df to deal with data types
    '''
    
    headers = df.columns
    first_row = []

    for col in headers:
        first_row.append(df[col][0])
    
    dictionary = dict( zip( headers, first_row) )

    return dictionary

In [20]:
# applying headers_and_first_row to df_lounge_eligibility
format_df = headers_and_first_row(df_lounge_eligibility)
format_2 = headers_and_first_row(df_flight_info)

In [21]:
# Note that some columns have blank spaces!
format_df
format_2

{'GMT_PLND_DEP_TS': Timestamp('2023-04-30 20:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-04-30 20:34:00'),
 'OPG_ALN_CD': 'BA    ',
 'OPG_FLT_NO': 57,
 'ACT_DEP_STN_CD': 'LHR   ',
 'ACT_DEP_TML_CD': '5 ',
 'PLND_ARR_STN_CD': 'JNB   ',
 'ACT_ARR_STN_CD': 'JNB   ',
 'IATA_AC_TYP_CD': '388   ',
 'ACT_AC_TYP_CD': 'A8  ',
 'ROUTE': 'LHRJNB      '}

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 1:** Pre-process **all** the tables above (df_lounge_eligibility, df_flight_info, df_country, df_acft_typ).

</font>

</ol>
</div>

We will first strip all frames of spaces. We will then go through each frame to properly prepare it.

We will then load some additional datasets that will be useful for analysis.

In [22]:
# Stripping strings

tables = [df_lounge_eligibility,df_flight_info,df_country,df_acft_typ,df_additional_country_decodes]

# Define a function that fine all string fields and remove all blak spaces
def data_cleaning_string(df):
    # Get names of all fields in a dataframe
    fields = df.columns
    # loop for all fields, if data type is string then remove blank spaces
    for f in fields:
        if type(df[f][0]) == str:
            df[f] = df[f].str.strip()
    return df

for i,t in enumerate(tables):
    tables[i] = data_cleaning_string(t)

In [23]:
# Removing duplucates of flight_info

# Sort the table
df_flight_info = df_flight_info.sort_values(['GMT_PLND_DEP_TS','OPG_FLT_NO','GMT_ACT_DEP_TS'])
# Remove all duplicates and keep the first rows of all duplicated
df_flight_info = df_flight_info.drop_duplicates(subset = ['OPG_FLT_NO','GMT_PLND_DEP_TS'],keep = "first")


In [24]:
# 2. Flight info
# Ian's code to clean dataset - identify latest departure date for
# duplicated flights and eliminate it.
df_flight_info['GMT_PLND_DEP_DT'] = df_flight_info['GMT_PLND_DEP_TS'].dt.date
df_flight_info['GMT_PLND_DEP_TIME'] = df_flight_info['GMT_PLND_DEP_TS'].dt.time

In [25]:
# 3. Country
df_country['DEP_STN_CD'] = df_country['ROUTE'].str.slice(0,3)
df_country['ARR_STN_CD'] = df_country['ROUTE'].str.slice(3,6)


In [26]:
# 4. Aircraft type
df_acft_typ.sort_values('FIRST_SEATS_QTY').tail(10)

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
8,789,L8,WB,8,42,39,127
21,77W,F7,WB,8,76,40,130
34,777,R7,WB,10,52,40,134
29,777,W7,WB,12,48,32,127
12,77W,G7,WB,14,56,44,183
13,744,S4,WB,14,86,30,145
22,744,V4,WB,14,52,36,235
15,388,A8,WB,14,97,55,303
27,777,V7,WB,14,48,40,124
24,777,P7,WB,17,48,24,127


In [27]:
df_lounge_eligibility.shape

(1534750, 12)

<br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">**Exercise 2:** Join the tables below
   
    - df_flight_info
    - df_country
    - df_acft_typ
    
to the table df_lounge_eligibility to generate a final dataset.

</font>

</ol>
</div>

In [28]:
# df_lounge_eligibility.head(5)

In [29]:
df_lounge_elig_flight_info = pd.merge(df_lounge_eligibility,# left table
                                     df_flight_info, # right table
                                     left_on = ['OPERATING_AIRLINE_CD','OPERATING_FLT_NO','GMT_UPLIFT_DT'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['OPG_ALN_CD','OPG_FLT_NO','GMT_PLND_DEP_DT'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

df_lounge_elig_flight_info.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00


In [30]:
# your code here!
df_lounge_elig_flight_info.shape

(1534750, 25)

Get rows with any null values

In [31]:
df_lounge_elig_flight_info[df_lounge_elig_flight_info.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME
300,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,SAPP,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,2019-09-298598,BA,8598,2019-09-29,LHR,CFU,M,M,,,Not eligible,43,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1452,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,,,Not eligible,144,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11711,2019-06-168586,BA,8586,2019-06-16,LHR,KLX,M,M,Gold,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_flight_info[df_flight_info['OPG_FLT_NO'] == 8642]

,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME


### Merge `df_country` to table

In [33]:
[headers_and_first_row(d) for d in [df_lounge_elig_flight_info,df_country]]

[{'Skew_Id': '2023-08-05618',
  'OPERATING_AIRLINE_CD': 'BA',
  'OPERATING_FLT_NO': 618,
  'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
  'UPLIFT_STN_CD': 'LHR',
  'DISCHARGE_STN_CD': 'OLB',
  'BOOKED_CABIN_CD': 'C',
  'TRAVEL_CABIN_CD': 'C',
  'BA_PAX_TIER': 'Gold For Life',
  'ONEWORLD_TIER': 'EMER',
  'Lounge_eligibility_tier': 'Tier 2',
  'pax': 2,
  'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
  'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
  'OPG_ALN_CD': 'BA',
  'OPG_FLT_NO': 618.0,
  'ACT_DEP_STN_CD': 'LHR',
  'ACT_DEP_TML_CD': '3',
  'PLND_ARR_STN_CD': 'OLB',
  'ACT_ARR_STN_CD': 'OLB',
  'IATA_AC_TYP_CD': '319',
  'ACT_AC_TYP_CD': 'M4',
  'ROUTE': 'LHROLB',
  'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
  'GMT_PLND_DEP_TIME': datetime.time(14, 25)},
 {'ROUTE': 'LHRINV',
  'COUNTRY_CD': 'GB',
  'COUNTRY_NM': 'United Kingdom and Northern Ireland',
  'CORP_GEOG_CTRY_GRP_NM': 'UK',
  'CORP_GEOG_CONTINENT_NM': 'UK',
  'DEP_STN_CD': 'LHR',
  'ARR_STN_CD': 'INV'}]

In [34]:
df_additional_country_decodes.columns
df_country.columns

Index(['ROUTE', 'COUNTRY_CD', 'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM',
       'CORP_GEOG_CONTINENT_NM', 'DEP_STN_CD', 'ARR_STN_CD'],
      dtype='object')

In [35]:
# For efficiency, we should merge df_country and df_additional_country_decodes first.
# Firstly, organise df_additional_country_decodes to have the same columns.
# if statement to avoid throwing errors if column renaming from cell below has already been done.
if 'STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'STN_CD'
elif 'ARR_STN_CD' in df_additional_country_decodes.columns:
    station_code_col = 'ARR_STN_CD'
df_additional_country_decodes = df_additional_country_decodes[~df_additional_country_decodes[station_code_col].isin(df_country['ARR_STN_CD'])]
# for more efficiency, we can remove all those rows which are not needed as there are no lounge elig rows with that destinations.
df_additional_country_decodes = df_additional_country_decodes[df_additional_country_decodes[station_code_col].isin(df_lounge_elig_flight_info['DISCHARGE_STN_CD'])]
print(df_additional_country_decodes.shape)

(30, 5)


In [36]:
# We need df_additional_country_decodes to have the same column names and order as df_country, ready for pd.concat.
# By merging now, we eliminate the need to merge separately with df_lounge_elig.
df_additional_country_decodes['ROUTE'] = 'LHR' + df_additional_country_decodes['STN_CD']
df_additional_country_decodes['DEP_STN_CD'] = 'LHR'
# We must ensure column names and order of df_additional_country_decodes match df_country.
df_additional_country_decodes.rename(columns={'STN_CD':'ARR_STN_CD'},inplace=True)
df_additional_country_decodes = df_additional_country_decodes[df_country.columns]

In [37]:
# concat together.
df_country = pd.concat([df_country,df_additional_country_decodes])

In [38]:
df_lounge_elig_flight_info_country = pd.merge(df_lounge_elig_flight_info,# left table
                                     df_country, # right table
                                     left_on = ['DISCHARGE_STN_CD'], # left on? e.g. which columns from the left table are you joining on to?
                                     right_on = ['ARR_STN_CD'] , # right on? # left on? e.g. which columns from the right table are you joining on to?
                                     how = "left" # how? e.g. left, right, inner,etc
                                     )

# df_lounge_elig_flight_info = df_lounge_elig_flight_info.drop_duplicates()

df_lounge_elig_flight_info_country.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI


In [39]:
df_lounge_elig_flight_info_country.shape

(1534750, 32)

Check for null values

In [40]:
df_lounge_elig_flight_info_country[df_lounge_elig_flight_info_country['CORP_GEOG_CONTINENT_NM'].isna()].head(5)
# There are no null values arising from lack of country data.

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD


In [41]:
# to avoid breaking anything, we first remove all values we do not need.
# Firstly, remove those values in df_country.
# then remove values not in df_lounge_elig.
df_additional_country_decodes = df_additional_country_decodes[~df_additional_country_decodes['STN_CD'].isin(df_country['ARR_STN_CD'])]
df_additional_country_decodes = df_additional_country_decodes[df_additional_country_decodes['STN_CD'].isin(df_lounge_elig_flight_info_country['DISCHARGE_STN_CD'])]
df_additional_country_decodes.head(40)
#df_lounge_elig_flight_info_country.merge(df_additional_country_decodes)

KeyError: 'STN_CD'

### `df_acft_typ`

In [41]:
df_lounge_elig_flight_info_country_acft_typ = pd.merge(
    df_lounge_elig_flight_info_country,
    df_acft_typ,
    left_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    right_on = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD'],
    how = 'left'
)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,0.0,45.0,0.0,212.0


In [42]:
df_lounge_elig_flight_info_country_acft_typ.shape

(1534750, 37)

Check for null

In [43]:
df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ.isna().any(axis=1)].head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
300,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,SAPP,Tier 3,2,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
466,2019-09-298598,BA,8598,2019-09-29,LHR,CFU,M,M,,,Not eligible,43,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRCFU,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,CFU,NaN,NaN,NaN,NaN,NaN
876,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,Silver,,Tier 3,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
1452,2019-09-148642,BA,8642,2019-09-14,LHR,ADB,M,M,,,Not eligible,144,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRADB,TR,Turkey,MEDITERRANEAN,EUROPE EXC UK,LHR,ADB,NaN,NaN,NaN,NaN,NaN
11711,2019-06-168586,BA,8586,2019-06-16,LHR,KLX,M,M,Gold,EMER,Tier 2,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LHRKLX,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,KLX,NaN,NaN,NaN,NaN,NaN


In [44]:
df_acft_typ[df_acft_typ['ACT_AC_TYP_CD'] == 'M4']

,IATA_AC_TYP_CD,ACT_AC_TYP_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
20,319,M4,NB,0,21,0,113


### More cleaning for columns redundancy

In [45]:
headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

{'Skew_Id': '2023-08-05618',
 'OPERATING_AIRLINE_CD': 'BA',
 'OPERATING_FLT_NO': 618,
 'GMT_UPLIFT_DT': datetime.date(2023, 8, 5),
 'UPLIFT_STN_CD': 'LHR',
 'DISCHARGE_STN_CD': 'OLB',
 'BOOKED_CABIN_CD': 'C',
 'TRAVEL_CABIN_CD': 'C',
 'BA_PAX_TIER': 'Gold For Life',
 'ONEWORLD_TIER': 'EMER',
 'Lounge_eligibility_tier': 'Tier 2',
 'pax': 2,
 'GMT_PLND_DEP_TS': Timestamp('2023-08-05 14:25:00'),
 'GMT_ACT_DEP_TS': Timestamp('2023-08-05 14:57:00'),
 'OPG_ALN_CD': 'BA',
 'OPG_FLT_NO': 618.0,
 'ACT_DEP_STN_CD': 'LHR',
 'ACT_DEP_TML_CD': '3',
 'PLND_ARR_STN_CD': 'OLB',
 'ACT_ARR_STN_CD': 'OLB',
 'IATA_AC_TYP_CD': '319',
 'ACT_AC_TYP_CD': 'M4',
 'ROUTE_x': 'LHROLB',
 'GMT_PLND_DEP_DT': datetime.date(2023, 8, 5),
 'GMT_PLND_DEP_TIME': datetime.time(14, 25),
 'ROUTE_y': 'LHROLB',
 'COUNTRY_CD': 'IT',
 'COUNTRY_NM': 'Italy',
 'CORP_GEOG_CTRY_GRP_NM': 'MEDITERRANEAN',
 'CORP_GEOG_CONTINENT_NM': 'EUROPE EXC UK',
 'DEP_STN_CD': 'LHR',
 'ARR_STN_CD': 'OLB',
 'WB_NB_CAT': 'NB',
 'FIRST_SEATS_QTY': 0.0

In [46]:
# to_drop = ['GMT_PLND_DEP_TS','GMT_ACT_DEP_TS','OPG_ALN_CD','OPG_FLT_NO','ACT_DEP_STN_CD','PLND_ARR_STN_CD','ACT_ARR_STN_CD','GMT_UPLIFT_DT']

# df_lounge_elig_flight_info_country_acft_typ = df_lounge_elig_flight_info_country_acft_typ.drop(columns = to_drop,)

df_lounge_elig_flight_info_country_acft_typ.head(5)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,0.0,21.0,0.0,113.0
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,0.0,21.0,0.0,113.0
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,0.0,20.0,0.0,150.0
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,0.0,16.0,0.0,119.0
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,0.0,45.0,0.0,212.0


Add new row `'GMT_UPLIFT_YR'`

In [47]:
df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_YR'] = [dts.year for dts in df_lounge_elig_flight_info_country_acft_typ['GMT_UPLIFT_DT']]

df_lounge_elig_flight_info_country_acft_typ.head(10)

,Skew_Id,OPERATING_AIRLINE_CD,OPERATING_FLT_NO,GMT_UPLIFT_DT,UPLIFT_STN_CD,DISCHARGE_STN_CD,BOOKED_CABIN_CD,TRAVEL_CABIN_CD,BA_PAX_TIER,ONEWORLD_TIER,Lounge_eligibility_tier,pax,GMT_PLND_DEP_TS,GMT_ACT_DEP_TS,OPG_ALN_CD,OPG_FLT_NO,ACT_DEP_STN_CD,ACT_DEP_TML_CD,PLND_ARR_STN_CD,ACT_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,ROUTE_x,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,ROUTE_y,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,DEP_STN_CD,ARR_STN_CD,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR
0,2023-08-05618,BA,618,2023-08-05,LHR,OLB,C,C,Gold For Life,EMER,Tier 2,2,2023-08-05 14:25:00,2023-08-05 14:57:00,BA,618.0,LHR,3,OLB,OLB,319,M4,LHROLB,2023-08-05,14:25:00,LHROLB,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,OLB,NB,0.0,21.0,0.0,113.0,2023
1,2023-05-17920,BA,920,2023-05-17,LHR,STR,C,C,,,Tier 3,13,2023-05-17 16:35:00,2023-05-17 16:43:00,BA,920.0,LHR,3,STR,STR,319,M4,LHRSTR,2023-05-17,16:35:00,LHRSTR,DE,Germany,WEST EUROPE,EUROPE EXC UK,LHR,STR,NB,0.0,21.0,0.0,113.0,2023
2,2019-04-15770,BA,770,2019-04-15,LHR,OSL,M,M,,,Not eligible,75,2019-04-15 18:25:00,2019-04-15 18:52:00,BA,770.0,LHR,5,OSL,OSL,32A,H3,LHROSL,2019-04-15,18:25:00,LHROSL,NO,Norway,SCANDINAVIA,EUROPE EXC UK,LHR,OSL,NB,0.0,20.0,0.0,150.0,2019
3,2019-09-23348,BA,348,2019-09-23,LHR,NCE,M,M,,EMER,Tier 2,6,2019-09-23 14:40:00,2019-09-23 14:45:00,BA,348.0,LHR,5,NCE,NCE,319,A4,LHRNCE,2019-09-23,14:40:00,LHRNCE,FR,France,WEST EUROPE,EUROPE EXC UK,LHR,NCE,NB,0.0,16.0,0.0,119.0,2019
4,2019-09-16155,BA,155,2019-09-16,LHR,CAI,J,J,,,Tier 3,23,2019-09-16 16:15:00,2019-09-16 16:45:00,BA,155.0,LHR,5,CAI,CAI,343,W9,LHRCAI,2019-09-16,16:15:00,LHRCAI,EG,Egypt,NORTH AFRICA,AFRICA,LHR,CAI,WB,0.0,45.0,0.0,212.0,2019
5,2023-08-05666,BA,666,2023-08-05,LHR,CHQ,C,C,Gold,EMER,Tier 2,3,2023-08-05 09:45:00,2023-08-05 09:56:00,BA,666.0,LHR,5,CHQ,CHQ,32N,N3,LHRCHQ,2023-08-05,09:45:00,LHRCHQ,GR,Greece,MEDITERRANEAN,EUROPE EXC UK,LHR,CHQ,NB,0.0,20.0,0.0,150.0,2023
6,2019-07-03227,BA,227,2019-07-03,LHR,ATL,J,J,Gold,EMER,Tier 2,4,2019-07-03 15:00:00,2019-07-03 15:26:00,BA,227.0,LHR,5,ATL,ATL,777,V7,LHRATL,2019-07-03,15:00:00,LHRATL,US,United States,USA,NORTH AMERICA,LHR,ATL,WB,14.0,48.0,40.0,124.0,2019
7,2019-06-301448,BA,1448,2019-06-30,LHR,EDI,C,C,,SAPP,Tier 3,4,2019-06-30 14:20:00,2019-06-30 14:21:00,BA,1448.0,LHR,5,EDI,EDI,320,A3,LHREDI,2019-06-30,14:20:00,LHREDI,GB,United Kingdom and Northern Ireland,UK,UK,LHR,EDI,NB,0.0,24.0,0.0,132.0,2019
8,2019-04-27886,BA,886,2019-04-27,LHR,OTP,M,M,,EMER,Tier 2,1,2019-04-27 12:50:00,2019-04-27 13:47:00,BA,886.0,LHR,5,OTP,OTP,32N,N3,LHROTP,2019-04-27,12:50:00,LHROTP,RO,Romania,EASTERN EUROPE,EUROPE EXC UK,LHR,OTP,NB,0.0,20.0,0.0,150.0,2019
9,2023-10-01558,BA,558,2023-10-01,LHR,FCO,M,M,Gold,EMER,Tier 2,3,2023-10-01 18:15:00,2023-10-01 18:23:00,BA,558.0,LHR,5,FCO,FCO,320,R3,LHRFCO,2023-10-01,18:15:00,LHRFCO,IT,Italy,MEDITERRANEAN,EUROPE EXC UK,LHR,FCO,NB,0.0,28.0,0.0,139.0,2023


Create Pivot Table

In [176]:
df = pd.DataFrame(df_lounge_elig_flight_info_country_acft_typ)

selected_columns = ['Skew_Id', 'OPERATING_FLT_NO', 'GMT_UPLIFT_DT', 'BOOKED_CABIN_CD', 'TRAVEL_CABIN_CD', 'BA_PAX_TIER', 'ONEWORLD_TIER', 'Lounge_eligibility_tier', 'pax', 'GMT_PLND_DEP_TS',  'OPG_FLT_NO','ACT_DEP_TML_CD', 'PLND_ARR_STN_CD', 'IATA_AC_TYP_CD', 'ACT_AC_TYP_CD', 'GMT_PLND_DEP_DT', 'GMT_PLND_DEP_TIME', 'COUNTRY_CD', 'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM', 'CORP_GEOG_CONTINENT_NM','WB_NB_CAT', 'FIRST_SEATS_QTY', 'CLUB_SEATS_QTY', 'PREM_ECONOMY_SEATS_QTY', 'ECONOMY_SEATS_QTY', 'GMT_UPLIFT_YR']
new_df = df.loc[:, selected_columns]

# Pivot the DataFrame using pivot_table
pivot_df = pd.pivot_table(new_df, values='pax', index=['Skew_Id', 'OPERATING_FLT_NO', 'GMT_UPLIFT_DT', 'GMT_PLND_DEP_TS', 'OPG_FLT_NO','ACT_DEP_TML_CD', 'PLND_ARR_STN_CD', 'IATA_AC_TYP_CD', 'ACT_AC_TYP_CD', 'GMT_PLND_DEP_DT', 'GMT_PLND_DEP_TIME', 'COUNTRY_CD', 'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM', 'CORP_GEOG_CONTINENT_NM','WB_NB_CAT', 'FIRST_SEATS_QTY', 'CLUB_SEATS_QTY', 'PREM_ECONOMY_SEATS_QTY', 'ECONOMY_SEATS_QTY', 'GMT_UPLIFT_YR'],
                          columns='Lounge_eligibility_tier', aggfunc='sum', fill_value=0).reset_index()

pivot_df['Total_Passesgers'] = pivot_df['Not eligible'] + pivot_df['Tier 1'] + pivot_df['Tier 2'] + pivot_df['Tier 3']
pivot_df['Not eligible ratio'] = pivot_df['Not eligible']/pivot_df['Total_Passesgers']
pivot_df['Tier 1 ratio'] = pivot_df['Tier 1']/pivot_df['Total_Passesgers']
pivot_df['Tier 2 ratio'] = pivot_df['Tier 2']/pivot_df['Total_Passesgers']
pivot_df['Tier 3 ratio'] = pivot_df['Tier 3']/pivot_df['Total_Passesgers']



# Display the resulting DataFrame
first_10_column_names = pivot_df.columns[:10].tolist()
print(first_10_column_names)

print(pivot_df.columns)

pivot_df.head(10)

['Skew_Id', 'OPERATING_FLT_NO', 'GMT_UPLIFT_DT', 'GMT_PLND_DEP_TS', 'OPG_FLT_NO', 'ACT_DEP_TML_CD', 'PLND_ARR_STN_CD', 'IATA_AC_TYP_CD', 'ACT_AC_TYP_CD', 'GMT_PLND_DEP_DT']
Index(['Skew_Id', 'OPERATING_FLT_NO', 'GMT_UPLIFT_DT', 'GMT_PLND_DEP_TS',
       'OPG_FLT_NO', 'ACT_DEP_TML_CD', 'PLND_ARR_STN_CD', 'IATA_AC_TYP_CD',
       'ACT_AC_TYP_CD', 'GMT_PLND_DEP_DT', 'GMT_PLND_DEP_TIME', 'COUNTRY_CD',
       'COUNTRY_NM', 'CORP_GEOG_CTRY_GRP_NM', 'CORP_GEOG_CONTINENT_NM',
       'WB_NB_CAT', 'FIRST_SEATS_QTY', 'CLUB_SEATS_QTY',
       'PREM_ECONOMY_SEATS_QTY', 'ECONOMY_SEATS_QTY', 'GMT_UPLIFT_YR',
       'Not eligible', 'Tier 1', 'Tier 2', 'Tier 3', 'Total_Passesgers',
       'Not eligible ratio', 'Tier 1 ratio', 'Tier 2 ratio', 'Tier 3 ratio'],
      dtype='object', name='Lounge_eligibility_tier')


Lounge_eligibility_tier,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TS,OPG_FLT_NO,ACT_DEP_TML_CD,PLND_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passesgers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio
0,2019-03-31103,103,2019-03-31,2019-03-31 17:30:00,103.0,5,YYC,788,B8,2019-03-31,17:30:00,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513
1,2019-03-31105,105,2019-03-31,2019-03-31 21:25:00,105.0,5,DXB,777,V7,2019-03-31,21:25:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,137,13,13,46,209,0.655502,0.062201,0.062201,0.220096
2,2019-03-31107,107,2019-03-31,2019-03-31 11:50:00,107.0,5,DXB,777,V7,2019-03-31,11:50:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,126,11,18,46,201,0.626866,0.054726,0.089552,0.228856
3,2019-03-31109,109,2019-03-31,2019-03-31 20:15:00,109.0,5,DXB,777,V7,2019-03-31,20:15:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,139,14,22,42,217,0.640553,0.064516,0.101382,0.193548
4,2019-03-3111,11,2019-03-31,2019-03-31 18:55:00,11.0,5,SIN,388,A8,2019-03-31,18:55:00,SG,Singapore,FAR EAST JSA HUBS,AUSTRALASIA PACIFIC,WB,14.0,97.0,55.0,303.0,2019,308,17,43,75,443,0.695260,0.038375,0.097065,0.169300
5,2019-03-31113,113,2019-03-31,2019-03-31 15:15:00,113.0,5,JFK,777,W7,2019-03-31,15:15:00,US,United States,USA,NORTH AMERICA,WB,12.0,48.0,32.0,127.0,2019,135,12,22,43,212,0.636792,0.056604,0.103774,0.202830
6,2019-03-31115,115,2019-03-31,2019-03-31 13:30:00,115.0,5,JFK,777,V7,2019-03-31,13:30:00,US,United States,USA,NORTH AMERICA,WB,14.0,48.0,40.0,124.0,2019,132,14,26,40,212,0.622642,0.066038,0.122642,0.188679
7,2019-03-31117,117,2019-03-31,2019-03-31 07:30:00,117.0,5,JFK,744,S4,2019-03-31,07:30:00,US,United States,USA,NORTH AMERICA,WB,14.0,86.0,30.0,145.0,2019,138,19,16,61,234,0.589744,0.081197,0.068376,0.260684
8,2019-03-31119,119,2019-03-31,2019-03-31 13:15:00,119.0,5,BLR,77W,G7,2019-03-31,13:15:00,IN,India,SOUTH ASIA,ASIA SOUTH EAST AND CENTRAL,WB,14.0,56.0,44.0,183.0,2019,204,3,12,42,261,0.781609,0.011494,0.045977,0.160920
9,2019-03-31123,123,2019-03-31,2019-03-31 18:45:00,123.0,5,DOH,789,L8,2019-03-31,18:45:00,QA,Qatar,SOUTHERN GULF,MIDDLE EAST,WB,8.0,42.0,39.0,127.0,2019,133,6,11,27,177,0.751412,0.033898,0.062147,0.152542


Save as CSV

In [49]:
df_lounge_elig_flight_info_country_acft_typ.to_csv('Merged.csv')
pivot_df.to_csv('TiersTallyPerFlight.csv')

<br><br>
<div class="alert alert-" style = "border-radius:10px;border-width:3px;border-color:lightblue;font-family:Verdana,sans-serif;font-size:16px;">

<font size="3">Exercise 3: Based on S2019/S2023 data, provide a lookup table of Lounge eligibility assumptions that can be applied to a future schedule. To do so, answer each of the following questions in the Markdown cell provided below. 

- What level of granularity do you use?
- What metric do you use to come up with Lounge eligibility profiles?

    
Note 1: **Provide evidence for your assumptions.** This can be in the form of tables, graphs, correlation matrix, etc.
    
Note 2: Make use of the examples below to give structure to your answer. Feel free to attend the Hackathon Clinics if you have any questions. 
</font>


    
</ol>
</div>

**Reasoning (Example 1)**:


Assume:
- Data has been preprocessed.
- Data has been joined, and a final dataset has been created. This dataset is the result of joining the 4 tables.

The final dataset has been called `df_lounge_elig_flight_info_country_acft_typ`.

<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their aircraft type. I will therefore have a lookup table with two categories: Narrowbody and Widebody. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by Aircraft Type, by Tier. Then I have divided them by the total number of passengers by Aircraft Type. For example:
    - For NB aircrafts, and for Tier 1 passengers: In S19 we had 41,728 pax eligible for Tier 1 out of 7,222,830 pax flying on Narrowbody aircraft. This represents 0.6% of the costumers and I assume that this will be the number of costumers elegible for this specific Lounge in a future schedule.   

In [50]:
# your code here!

In [51]:
# columns that you want to group by
list_groupby = ['WB_NB_CAT','Lounge_eligibility_tier']

# grouping by WB_NB_CAT and Lounge_eligibility_tier
df_groupby_wb_nb = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

In [52]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count
0,NB,Not eligible,9741188
1,NB,Tier 1,53292
2,NB,Tier 2,744829
3,NB,Tier 3,2240579
4,WB,Not eligible,5730922
5,WB,Tier 1,236775
6,WB,Tier 2,391264
7,WB,Tier 3,1523611


In [53]:
# getting the number of pax by aircraft type

# columns that you want to group by
list_groupby = ['WB_NB_CAT']

# grouping by WB_NB_CAT
df_groupby_wb_nb_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [54]:
# a look at the data
df_groupby_wb_nb_ttl

,WB_NB_CAT,pax_count_ttl
0,NB,12779888
1,WB,7882572


In [55]:
# now, let's left join df_groupby_wb_nb_ttl onto df_groupby_wb_nb
# this adds a new column to the df_groupby_wb_nb table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_wb_nb = pd.merge(df_groupby_wb_nb,
                            df_groupby_wb_nb_ttl,
                            on = ['WB_NB_CAT'],
                            how = 'left'
                           )

In [56]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,NB,Not eligible,9741188,12779888
1,NB,Tier 1,53292,12779888
2,NB,Tier 2,744829,12779888
3,NB,Tier 3,2240579,12779888
4,WB,Not eligible,5730922,7882572
5,WB,Tier 1,236775,7882572
6,WB,Tier 2,391264,7882572
7,WB,Tier 3,1523611,7882572


In [57]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_wb_nb['pax_eligible%'] = (df_groupby_wb_nb['pax_count'] / df_groupby_wb_nb['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_wb_nb['pax_eligible%'] = df_groupby_wb_nb['pax_eligible%'].map('{:,.1f}%'.format)

In [58]:
# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_wb_nb.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [59]:
# a look at the data
df_groupby_wb_nb

,WB_NB_CAT,Lounge_eligibility_tier,pax_eligible%
0,NB,Not eligible,76.2%
1,NB,Tier 1,0.4%
2,NB,Tier 2,5.8%
3,NB,Tier 3,17.5%
4,WB,Not eligible,72.7%
5,WB,Tier 1,3.0%
6,WB,Tier 2,5.0%
7,WB,Tier 3,19.3%


In [60]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_wb_nb['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_wb_nb = df_groupby_wb_nb[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [61]:
# using set_index to come up with the final lookup table
df_groupby_wb_nb = df_groupby_wb_nb.set_index(['WB_NB_CAT','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [62]:
# final table
df_groupby_wb_nb

pax_eligible%              
Lounge_eligibility_tier        Tier 1 Tier 2 Tier 3
WB_NB_CAT                                          
NB                               0.4%   5.8%  17.5%
WB                               3.0%   5.0%  19.3%

<br>

**Feedback:** This analysis provides a lookup table in the format needed to be input into a future schedule. Nevertheless, this analysis is too high level and you haven't provided any evidence for your assumptions. To further enhance your answer use insights from the data and provide evidence for your assumptions. Please find some ideas below: 

- Using the same categories (WB,NB), plot data overtime to better understand the peaks for the different lounges. 
- Is there a way to split Widebody into more categories? Do the Haul, Region, Time of Day, or Country play a role in the number of passengers that are eligible in Tier 1, Tier 2 and Tier 3? Etc...

<br><br>

**Reasoning (Example 2)**:



<u>What level of granularity do you use?</u>
- I have decided to split all flights in the network based on their flight number. I will therefore have a lookup table with a lot of categories as each flight number is a category. 

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- I have sumed up all the passengers by flight number, by Tier. Then I have divided them by the total number of passengers flight number.  

In [63]:
# your code here!

In [64]:
# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_flt_no = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count
0,5,HND,Not eligible,15825
1,5,HND,Tier 1,220
2,5,HND,Tier 2,888
3,5,HND,Tier 3,4469


In [65]:
# getting the number of pax by OPERATING_FLT_NO and DISCHARGE_STN_CD

# columns that you want to group by
list_groupby = ['OPERATING_FLT_NO','DISCHARGE_STN_CD']

# grouping by WB_NB_CAT
df_groupby_flt_no_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

In [66]:
# a look at the data
df_groupby_flt_no_ttl.head(2)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,pax_count_ttl
0,5,HND,21402
1,5,NRT,38378


In [67]:
# now, let's left join df_groupby_flt_no_ttl onto df_groupby_flt_no
# this adds a new column to the df_groupby_flt_no table (pax_count_ttl) that will be used to get the percentage of passenger eligible by Tier

df_groupby_flt_no = pd.merge(df_groupby_flt_no,
                            df_groupby_flt_no_ttl,
                            on = ['OPERATING_FLT_NO','DISCHARGE_STN_CD'],
                            how = 'left'
                           )

In [68]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,5,HND,Not eligible,15825,21402
1,5,HND,Tier 1,220,21402
2,5,HND,Tier 2,888,21402
3,5,HND,Tier 3,4469,21402


In [69]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_flt_no['pax_eligible%'] = (df_groupby_flt_no['pax_count'] / df_groupby_flt_no['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_flt_no['pax_eligible%'] = df_groupby_flt_no['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_flt_no.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [70]:
# a look at the data
df_groupby_flt_no.head(4)

,OPERATING_FLT_NO,DISCHARGE_STN_CD,Lounge_eligibility_tier,pax_eligible%
0,5,HND,Not eligible,73.9%
1,5,HND,Tier 1,1.0%
2,5,HND,Tier 2,4.1%
3,5,HND,Tier 3,20.9%


In [71]:
# dropping 'not eligible' rows - not needed anymore
mask = df_groupby_flt_no['Lounge_eligibility_tier'] == 'Not eligible'

df_groupby_flt_no = df_groupby_flt_no[~mask].copy()

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.

In [72]:
# using set_index to come up with the final lookup table
df_groupby_flt_no = df_groupby_flt_no.set_index(['OPERATING_FLT_NO','DISCHARGE_STN_CD','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

In [73]:
# final table
df_groupby_flt_no.head(4)

pax_eligible%              
Lounge_eligibility_tier                  Tier 1 Tier 2 Tier 3
OPERATING_FLT_NO DISCHARGE_STN_CD                            
5                HND                       1.0%   4.1%  20.9%
                 NRT                       3.6%   4.8%  19.1%
7                HND                       3.3%   5.2%  19.8%
9                BKK                       0.3%   4.2%  15.9%

<br>

**Feedback:** This analysis goes at a very granular level, we might come up with missing values if we apply this lounge eligibility profiles to a future schedule. Here are some ideas to further enhance your answer: 

- What would happen if we fly to a new destination in the future? How do we ensure we have a lounge eligibility profile for this new route?
- As you can see in the example above: Pax eligible for Tier 1 for the BKK flight is significantly different from the rest. What's the most used aircraft type for this route? And why it differs that much from the rest? Is it because of the route characteristics instead?
- For SH routes, we might change the time of departure for a specific flight number from one year to the next. Explore the possibility of using a clasification that takes that into account, like using Time of Day instead of flight number. 

### Ian  

### Linear Regression

In [74]:
# import numpy as np
# from sklearn import datasets, linear_model
# import matplotlib.pyplot as plt

In [75]:
# headers_and_first_row(df_lounge_elig_flight_info_country_acft_typ)

Separate the large tables to different tiers

In [76]:
# df_merged_T1 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 1'].reset_index()
# df_merged_T2 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 2'].reset_index()
# df_merged_T3 = df_lounge_elig_flight_info_country_acft_typ[df_lounge_elig_flight_info_country_acft_typ['Lounge_eligibility_tier'] == 'Tier 3'].reset_index()

# df_merged_T1 = df_merged_T1.rename(columns={'pax':'Tier_1_pax'})
# df_merged_T2 = df_merged_T2.rename(columns={'pax':'Tier_2_pax'})
# df_merged_T3 = df_merged_T3.rename(columns={'pax':'Tier_3_pax'})

In [77]:
# df_merged_T3.columns
# headers_and_first_row(df_merged_T1)


In [78]:
# y = np.array([1,2,3])
# x = np.array([-1,0,1])

# # y = y.to_numpy()
# # x = x.to_numpy().reshape(-1, 1)
# x = x.reshape(-1,1)

# # print(x.shape, y.shape)

# regr = linear_model.LinearRegression()
# regr.fit(x,y)

# regr.coef_, regr.intercept_, regr.rank_

### Compare by Year

In [79]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year

,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_count
0,2019,Not eligible,8147159
1,2019,Tier 1,196564
2,2019,Tier 2,635765
3,2019,Tier 3,1996226
4,2023,Not eligible,7372440
5,2023,Tier 1,93555
6,2023,Tier 2,501044
7,2023,Tier 3,1770087


In [80]:
# columns that you want to group by
list_groupby = ['GMT_UPLIFT_YR']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

# a look at the data
df_groupby_year_ttl

,GMT_UPLIFT_YR,pax_count_ttl
0,2019,10975714
1,2023,9737126


In [81]:
df_groupby_year = pd.merge(df_groupby_year,
                            df_groupby_year_ttl,
                            on = ['GMT_UPLIFT_YR'],
                            how = 'left'
                           )

df_groupby_year

,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_count,pax_count_ttl
0,2019,Not eligible,8147159,10975714
1,2019,Tier 1,196564,10975714
2,2019,Tier 2,635765,10975714
3,2019,Tier 3,1996226,10975714
4,2023,Not eligible,7372440,9737126
5,2023,Tier 1,93555,9737126
6,2023,Tier 2,501044,9737126
7,2023,Tier 3,1770087,9737126


In [82]:
# getting the percentage of pax elegible for each of the Tiers

df_groupby_year['pax_eligible%'] = (df_groupby_year['pax_count'] / df_groupby_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_year['pax_eligible%'] = df_groupby_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

In [83]:
df_groupby_year = df_groupby_year[df_groupby_year['Lounge_eligibility_tier'] != "Not eligible"]

df_groupby_year

,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_eligible%
1,2019,Tier 1,1.8%
2,2019,Tier 2,5.8%
3,2019,Tier 3,18.2%
5,2023,Tier 1,1.0%
6,2023,Tier 2,5.1%
7,2023,Tier 3,18.2%


Not much difference

### Compare by location and year

In [84]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_country_group_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR']

df_groupby_country_group_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_country_group_year = pd.merge(df_groupby_country_group_year,
                                         df_groupby_country_group_year_ttl,
                                         on = list_groupby,
                                         how = "left")

df_groupby_country_group_year = df_groupby_country_group_year[df_groupby_country_group_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_country_group_year['pax_eligible%'] = (df_groupby_country_group_year['pax_count'] / df_groupby_country_group_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_country_group_year['pax_eligible%'] = df_groupby_country_group_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_country_group_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_country_group_year.head(6)


,CORP_GEOG_CONTINENT_NM,CORP_GEOG_CTRY_GRP_NM,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_eligible%
1,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 1,3.7%
2,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 2,4.0%
3,AFRICA,EAST AND CENTRAL AFRICA,2019,Tier 3,17.6%
5,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 1,0.2%
6,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 2,4.3%
7,AFRICA,EAST AND CENTRAL AFRICA,2023,Tier 3,16.0%


In [85]:
df_groupby_country_group_year = df_groupby_country_group_year.set_index(['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_country_group_year

pax_eligible%  \
Lounge_eligibility_tier                                                  Tier 1   
CORP_GEOG_CONTINENT_NM      CORP_GEOG_CTRY_GRP_NM   GMT_UPLIFT_YR                 
AFRICA                      EAST AND CENTRAL AFRICA 2019                   3.7%   
                                                    2023                   0.2%   
                            NORTH AFRICA            2019                   0.5%   
                                                    2023                   0.3%   
                            SOUTHERN AFRICA         2019                   3.8%   
                                                    2023                   2.4%   
                            WEST AFRICA             2019                   3.7%   
                                                    2023                   2.3%   
ASIA FAR EAST               CHINA                   2019                   4.6%   
                                                    2023                   0.2%   
                            FAR EAST HUBS           2019                   3.8%   
                                                    2023                   1.4%   
                            FAR EAST OTHER          2019                   0.4%   
                            JAPAN                   2019                   3.5%   
                                                    2023                   1.0%   
ASIA SOUTH EAST AND CENTRAL SOUTH ASIA              2019                   2.8%   
                                                    2023                   0.4%   
AUSTRALASIA PACIFIC         FAR EAST JSA HUBS       2019                   2.3%   
                                                    2023                   3.4%   
EUROPE EXC UK               BENELUX                 2019                   0.7%   
                                                    2023                   0.5%   
                            EASTERN EUROPE          2019                   0.5%   
                                                    2023                   0.2%   
                            IRELAND RO              2019                   0.6%   
                                                    2023                   0.6%   
                            MEDITERRANEAN           2019                   0.7%   
                                                    2023                   0.3%   
                            SCANDINAVIA             2019                   0.6%   
                                                    2023                   0.4%   
                            WEST EUROPE             2019                   0.5%   
                                                    2023                   0.4%   
MIDDLE EAST                 MIDDLE EAST OTHER       2019                   0.3%   
                                                    2023                   0.1%   
                            NORTHERN GULF           2019                   5.0%   
                                                    2023                   3.0%   
                            SAUDI ARABIA            2019                   4.0%   
                                                    2023                   3.5%   
                            SOUTHERN GULF           2019                   4.8%   
                                                    2023                   3.6%   
NORTH AMERICA               CANADA                  2019                   2.3%   
                                                    2023                   0.2%   
                            NORTH AMERICA OTHER     2023                   3.6%   
                            USA                     2019                   4.3%   
                                                    2023                   2.4%   
SOUTH AMERICA INC CARIBBEAN CARIBBEAN               2019                    NaN   
                                                    2023                   2.7%   

### By continent

In [86]:
# columns that you want to group by
list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_cont_year = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count = ('pax','sum')
).reset_index()

list_groupby = ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR']

df_groupby_cont_year_ttl = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                pax_count_ttl = ('pax','sum')
).reset_index()

df_groupby_cont_year = pd.merge(df_groupby_cont_year,
                                df_groupby_cont_year_ttl,
                                on = list_groupby,
                                how = "left")

df_groupby_cont_year = df_groupby_cont_year[df_groupby_cont_year['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_cont_year['pax_eligible%'] = (df_groupby_cont_year['pax_count'] / df_groupby_cont_year['pax_count_ttl'] )*100 

# getting the pax_elegible% column in the right format
df_groupby_cont_year['pax_eligible%'] = df_groupby_cont_year['pax_eligible%'].map('{:,.1f}%'.format)

# dropping pax_count, pax_count_ttl columns - not needed anymore
df_groupby_cont_year.drop(columns=['pax_count','pax_count_ttl'],inplace = True)

df_groupby_cont_year.head(6)

,CORP_GEOG_CONTINENT_NM,GMT_UPLIFT_YR,Lounge_eligibility_tier,pax_eligible%
1,AFRICA,2019,Tier 1,3.5%
2,AFRICA,2019,Tier 2,4.8%
3,AFRICA,2019,Tier 3,18.5%
5,AFRICA,2023,Tier 1,1.7%
6,AFRICA,2023,Tier 2,4.3%
7,AFRICA,2023,Tier 3,18.2%


In [87]:
df_groupby_cont_year = df_groupby_cont_year.set_index(['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR','Lounge_eligibility_tier'],drop = True).unstack('Lounge_eligibility_tier')

df_groupby_cont_year

pax_eligible%              
Lounge_eligibility_tier                          Tier 1 Tier 2 Tier 3
CORP_GEOG_CONTINENT_NM      GMT_UPLIFT_YR                            
AFRICA                      2019                   3.5%   4.8%  18.5%
                            2023                   1.7%   4.3%  18.2%
ASIA FAR EAST               2019                   3.6%   4.8%  18.3%
                            2023                   1.0%   3.5%  18.3%
ASIA SOUTH EAST AND CENTRAL 2019                   2.8%   4.0%  19.1%
                            2023                   0.4%   3.0%  17.0%
AUSTRALASIA PACIFIC         2019                   2.3%   5.6%  18.3%
                            2023                   3.4%   7.4%  26.3%
EUROPE EXC UK               2019                   0.6%   6.4%  17.1%
                            2023                   0.4%   5.5%  18.1%
MIDDLE EAST                 2019                   4.0%   6.0%  20.2%
                            2023                   3.1%   5.0%  18.8%
NORTH AMERICA               2019                   4.1%   5.2%  20.5%
                            2023                   2.3%   5.1%  19.2%
SOUTH AMERICA INC CARIBBEAN 2019                   2.4%   3.6%  17.4%
                            2023                   1.5%   3.1%  17.3%
UK                          2019                   0.4%   5.7%  18.1%
                            2023                   0.4%   5.0%  16.8%

### By aircraft

In [88]:
list_groupby = ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY','Lounge_eligibility_tier']

# grouping by OPERATING_FLT_NO, DISCHARGE_STN_CD and Lounge_eligibility_tier
df_groupby_acft = df_lounge_elig_flight_info_country_acft_typ.groupby(list_groupby).agg(
                mean_pax_count = ('pax','mean')).reset_index()

df_groupby_acft['mean_pax_count'] = df_groupby_acft['mean_pax_count'].map('{:,.1f}'.format)

df_groupby_acft = df_groupby_acft[df_groupby_acft['Lounge_eligibility_tier'] != 'Not eligible']

df_groupby_acft = df_groupby_acft.set_index(list_groupby,drop = True).unstack('Lounge_eligibility_tier')

df_groupby_acft.fillna(0, inplace=True)

df_groupby_acft


mean_pax_count  \
Lounge_eligibility_tier                                                                                      Tier 1   
IATA_AC_TYP_CD ACT_AC_TYP_CD FIRST_SEATS_QTY CLUB_SEATS_QTY PREM_ECONOMY_SEATS_QTY ECONOMY_SEATS_QTY                  
319            A4            0.0             16.0           0.0                    119.0                        1.2   
               M4            0.0             21.0           0.0                    113.0                        1.1   
320            A3            0.0             24.0           0.0                    132.0                        1.2   
               E9            0.0             29.0           0.0                    130.0                        1.2   
               M2            0.0             24.0           0.0                    144.0                        1.1   
               R3            0.0             28.0           0.0                    139.0                        1.2   
               T3            0.0             26.0           0.0                    142.0                        1.2   
321            M6            0.0             23.0           0.0                    131.0                        1.3   
               V6            0.0             38.0           0.0                    160.0                        1.2   
               W6            0.0             42.0           0.0                    154.0                        1.3   
32A            H3            0.0             20.0           0.0                    150.0                        1.2   
32N            N3            0.0             20.0           0.0                    150.0                        1.2   
32Q            N6            0.0             32.0           0.0                    172.0                        1.2   
332            W9            0.0             30.0           21.0                   235.0                        1.0   
339            W9            0.0             30.0           21.0                   234.0                        1.1   
343            W9            0.0             45.0           0.0                    212.0                        1.3   
351            K5            0.0             56.0           56.0                   219.0                        1.3   
388            A8            14.0            97.0           55.0                   303.0                        2.2   
744            S4            14.0            86.0           30.0                   145.0                        2.1   
               V4            14.0            52.0           36.0                   235.0                        2.1   
777            D7            0.0             48.0           40.0                   184.0                        1.3   
               J7            0.0             32.0           48.0                   252.0                          0   
               K7            0.0             48.0           24.0                   203.0                          0   
               L7            0.0             48.0           24.0                   203.0                        1.3   
               M7            7.0             50.0           40.0                   138.0                        1.6   
               P7            17.0            48.0           24.0                   127.0                        2.1   
               R7            10.0            52.0           40.0                   134.0                        1.7   
               V7            14.0            48.0           40.0                   124.0                        2.1   
               W7            12.0            48.0           32.0                   127.0                        2.0   
77W            F7            8.0             76.0           40.0                   130.0                        1.7   
               G7            14.0            56.0           44.0                   183.0                        2.1   
781            X8            7.0             49.0      

## Same thing but using the Pivot Table

In [89]:
pivot_df.head(5)

Lounge_eligibility_tier,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TS,OPG_FLT_NO,ACT_DEP_TML_CD,PLND_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passesgers
0,2019-03-31103,103,2019-03-31,2019-03-31 17:30:00,103.0,5,YYC,788,B8,2019-03-31,17:30:00,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,154,1,9,33,197
1,2019-03-31105,105,2019-03-31,2019-03-31 21:25:00,105.0,5,DXB,777,V7,2019-03-31,21:25:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,137,13,13,46,209
2,2019-03-31107,107,2019-03-31,2019-03-31 11:50:00,107.0,5,DXB,777,V7,2019-03-31,11:50:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,126,11,18,46,201
3,2019-03-31109,109,2019-03-31,2019-03-31 20:15:00,109.0,5,DXB,777,V7,2019-03-31,20:15:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,139,14,22,42,217
4,2019-03-3111,11,2019-03-31,2019-03-31 18:55:00,11.0,5,SIN,388,A8,2019-03-31,18:55:00,SG,Singapore,FAR EAST JSA HUBS,AUSTRALASIA PACIFIC,WB,14.0,97.0,55.0,303.0,2019,308,17,43,75,443


Function for pivot tables

In [90]:
def group_pivot(df, list_groupby):
    Tiers = ['Not_Eligible','Tier_1','Tier_2','Tier_3']

    df_grouped = df.groupby(list_groupby).agg(
        Not_Eligible = ('Not eligible', sum),
        Tier_1 = ('Tier 1', sum),
        Tier_2 = ('Tier 2', sum),
        Tier_3 = ('Tier 3', sum),
        Total_Pax = ('Total_Passesgers', sum)).reset_index()

    for t in Tiers:
        df_grouped[t] = (df_grouped[t]/df_grouped['Total_Pax'])*100
        df_grouped[t] = df_grouped[t].map('{:,.2f}%'.format)

    return df_grouped

def group_pivot_to_lookup(df):
    return df.set_index(list(df.columns))

### By Terminal

In [91]:
df_pivot_groupby_term = group_pivot(pivot_df, ['ACT_DEP_TML_CD' ])
df_pivot_groupby_term

,ACT_DEP_TML_CD,Not_Eligible,Tier_1,Tier_2,Tier_3,Total_Pax
0,3,78.15%,1.16%,4.71%,15.99%,2592533
1,5,74.41%,1.44%,5.61%,18.54%,18069927


### By Year

In [92]:
df_pivot_groupby_year = group_pivot(pivot_df, ['GMT_UPLIFT_YR'])
df_pivot_groupby_year

,GMT_UPLIFT_YR,Not_Eligible,Tier_1,Tier_2,Tier_3,Total_Pax
0,2019,74.14%,1.80%,5.81%,18.25%,10927106
1,2023,75.71%,0.96%,5.15%,18.18%,9735354


### By country group and year

In [93]:
df_pivot_groupby_country_group_year = group_pivot(pivot_df, ['CORP_GEOG_CONTINENT_NM','CORP_GEOG_CTRY_GRP_NM','GMT_UPLIFT_YR'])
df_pivot_groupby_country_group_year.head(10)

,CORP_GEOG_CONTINENT_NM,CORP_GEOG_CTRY_GRP_NM,GMT_UPLIFT_YR,Not_Eligible,Tier_1,Tier_2,Tier_3,Total_Pax
0,AFRICA,EAST AND CENTRAL AFRICA,2019,74.77%,3.65%,4.01%,17.57%,67571
1,AFRICA,EAST AND CENTRAL AFRICA,2023,79.40%,0.24%,4.32%,16.04%,51668
2,AFRICA,NORTH AFRICA,2019,73.72%,0.47%,4.95%,20.86%,41975
3,AFRICA,NORTH AFRICA,2023,76.70%,0.26%,3.62%,19.43%,94028
4,AFRICA,SOUTHERN AFRICA,2019,70.66%,3.78%,5.33%,20.22%,220175
5,AFRICA,SOUTHERN AFRICA,2023,74.43%,2.41%,4.44%,18.72%,201263
6,AFRICA,WEST AFRICA,2019,75.77%,3.73%,4.48%,16.02%,158447
7,AFRICA,WEST AFRICA,2023,75.95%,2.34%,4.51%,17.20%,130417
8,ASIA FAR EAST,CHINA,2019,72.37%,4.63%,4.23%,18.77%,105600
9,ASIA FAR EAST,CHINA,2023,77.80%,0.25%,2.58%,19.37%,48677


In [94]:
group_pivot_to_lookup(df_pivot_groupby_country_group_year)

Empty DataFrame
Columns: []
Index: [(AFRICA, EAST AND CENTRAL AFRICA, 2019, 74.77%, 3.65%, 4.01%, 17.57%, 67571), (AFRICA, EAST AND CENTRAL AFRICA, 2023, 79.40%, 0.24%, 4.32%, 16.04%, 51668), (AFRICA, NORTH AFRICA, 2019, 73.72%, 0.47%, 4.95%, 20.86%, 41975), (AFRICA, NORTH AFRICA, 2023, 76.70%, 0.26%, 3.62%, 19.43%, 94028), (AFRICA, SOUTHERN AFRICA, 2019, 70.66%, 3.78%, 5.33%, 20.22%, 220175), (AFRICA, SOUTHERN AFRICA, 2023, 74.43%, 2.41%, 4.44%, 18.72%, 201263), (AFRICA, WEST AFRICA, 2019, 75.77%, 3.73%, 4.48%, 16.02%, 158447), (AFRICA, WEST AFRICA, 2023, 75.95%, 2.34%, 4.51%, 17.20%, 130417), (ASIA FAR EAST, CHINA, 2019, 72.37%, 4.63%, 4.23%, 18.77%, 105600), (ASIA FAR EAST, CHINA, 2023, 77.80%, 0.25%, 2.58%, 19.37%, 48677), (ASIA FAR EAST, FAR EAST HUBS, 2019, 72.29%, 3.84%, 5.35%, 18.52%, 174262), (ASIA FAR EAST, FAR EAST HUBS, 2023, 78.92%, 1.44%, 3.43%, 16.21%, 93144), (ASIA FAR EAST, FAR EAST OTHER, 2019, 80.35%, 0.36%, 3.79%, 15.50%, 39313), (ASIA FAR EAST, JAPAN, 2019, 73.10%, 3.53%, 4.79%, 18.57%, 111747), (ASIA FAR EAST, JAPAN, 2023, 73.78%, 1.02%, 4.39%, 20.81%, 54625), (ASIA SOUTH EAST AND CENTRAL, SOUTH ASIA, 2019, 74.03%, 2.84%, 4.05%, 19.09%, 286689), (ASIA SOUTH EAST AND CENTRAL, SOUTH ASIA, 2023, 79.55%, 0.43%, 3.02%, 17.01%, 298670), (AUSTRALASIA PACIFIC, FAR EAST JSA HUBS, 2019, 73.86%, 2.28%, 5.56%, 18.30%, 130531), (AUSTRALASIA PACIFIC, FAR EAST JSA HUBS, 2023, 62.91%, 3.44%, 7.37%, 26.28%, 43047), (EUROPE EXC UK, BENELUX, 2019, 72.67%, 0.74%, 8.31%, 18.28%, 375212), (EUROPE EXC UK, BENELUX, 2023, 77.70%, 0.50%, 6.34%, 15.46%, 288266), (EUROPE EXC UK, EASTERN EUROPE, 2019, 80.23%, 0.50%, 4.83%, 14.45%, 613867), (EUROPE EXC UK, EASTERN EUROPE, 2023, 80.21%, 0.24%, 4.22%, 15.33%, 460325), (EUROPE EXC UK, IRELAND RO, 2019, 75.53%, 0.60%, 6.64%, 17.23%, 202205), (EUROPE EXC UK, IRELAND RO, 2023, 75.84%, 0.64%, 6.34%, 17.19%, 126562), (EUROPE EXC UK, MEDITERRANEAN, 2019, 75.02%, 0.72%, 5.76%, 18.50%, 1210948), (EUROPE EXC UK, MEDITERRANEAN, 2023, 73.71%, 0.30%, 5.28%, 20.71%, 1419116), (EUROPE EXC UK, SCANDINAVIA, 2019, 76.71%, 0.57%, 6.70%, 16.03%, 530830), (EUROPE EXC UK, SCANDINAVIA, 2023, 77.04%, 0.39%, 6.03%, 16.53%, 375275), (EUROPE EXC UK, WEST EUROPE, 2019, 75.44%, 0.51%, 6.80%, 17.25%, 2613141), (EUROPE EXC UK, WEST EUROPE, 2023, 76.29%, 0.38%, 5.73%, 17.60%, 2327957), (MIDDLE EAST, MIDDLE EAST OTHER, 2019, 80.19%, 0.34%, 4.69%, 14.77%, 52779), (MIDDLE EAST, MIDDLE EAST OTHER, 2023, 78.34%, 0.14%, 3.62%, 17.90%, 22495), (MIDDLE EAST, NORTHERN GULF, 2019, 66.83%, 5.01%, 4.80%, 23.36%, 63033), (MIDDLE EAST, NORTHERN GULF, 2023, 76.24%, 3.03%, 2.96%, 17.78%, 53351), (MIDDLE EAST, SAUDI ARABIA, 2019, 65.71%, 4.03%, 6.63%, 23.63%, 65967), (MIDDLE EAST, SAUDI ARABIA, 2023, 67.20%, 3.51%, 7.76%, 21.53%, 31350), (MIDDLE EAST, SOUTHERN GULF, 2019, 69.43%, 4.78%, 6.58%, 19.22%, 154064), (MIDDLE EAST, SOUTHERN GULF, 2023, 72.14%, 3.62%, 5.50%, 18.73%, 112484), (NORTH AMERICA, CANADA, 2019, 76.61%, 2.34%, 3.54%, 17.52%, 247530), (NORTH AMERICA, CANADA, 2023, 79.10%, 0.23%, 3.40%, 17.28%, 191259), (NORTH AMERICA, NORTH AMERICA OTHER, 2023, 65.73%, 3.60%, 10.58%, 20.09%, 34024), (NORTH AMERICA, USA, 2019, 69.43%, 4.36%, 5.38%, 20.84%, 2109416), (NORTH AMERICA, USA, 2023, 72.97%, 2.44%, 5.19%, 19.40%, 1917147), (SOUTH AMERICA INC CARIBBEAN, CARIBBEAN, 2019, 72.58%, 0.00%, 7.26%, 20.16%, 124), (SOUTH AMERICA INC CARIBBEAN, CARIBBEAN, 2023, 73.51%, 2.66%, 3.67%, 20.17%, 39316), (SOUTH AMERICA INC CARIBBEAN, CENTRAL AMERICA, 2019, 73.91%, 3.92%, 4.25%, 17.93%, 35254), (SOUTH AMERICA INC CARIBBEAN, CENTRAL AMERICA, 2023, 76.70%, 3.56%, 2.54%, 17.20%, 33663), (SOUTH AMERICA INC CARIBBEAN, SOUTH AMERICA, 2019, 77.09%, 2.13%, 3.45%, 17.32%, 164725), (SOUTH AMERICA INC CARIBBEAN, SOUTH AMERICA, 2023, 80.05%, 0.58%, 3.01%, 16.36%, 123993), (UK, UK, 2019, 75.78%, 0.42%, 5.68%, 18.13%, 1151701), (UK, UK, 2023, 77.84%, 0.36%, 5.00%, 16.80%, 1163232)]

### By continent

In [95]:
df_pivot_groupby_cont_year = group_pivot(pivot_df, ['CORP_GEOG_CONTINENT_NM','GMT_UPLIFT_YR'])

group_pivot_to_lookup(df_pivot_groupby_cont_year)

Empty DataFrame
Columns: []
Index: [(AFRICA, 2019, 73.15%, 3.46%, 4.84%, 18.55%, 488168), (AFRICA, 2023, 75.83%, 1.73%, 4.28%, 18.15%, 477376), (ASIA FAR EAST, 2019, 73.26%, 3.64%, 4.79%, 18.32%, 430922), (ASIA FAR EAST, 2023, 77.21%, 1.03%, 3.49%, 18.27%, 196446), (ASIA SOUTH EAST AND CENTRAL, 2019, 74.03%, 2.84%, 4.05%, 19.09%, 286689), (ASIA SOUTH EAST AND CENTRAL, 2023, 79.55%, 0.43%, 3.02%, 17.01%, 298670), (AUSTRALASIA PACIFIC, 2019, 73.86%, 2.28%, 5.56%, 18.30%, 130531), (AUSTRALASIA PACIFIC, 2023, 62.91%, 3.44%, 7.37%, 26.28%, 43047), (EUROPE EXC UK, 2019, 75.82%, 0.58%, 6.44%, 17.17%, 5546203), (EUROPE EXC UK, 2023, 76.04%, 0.36%, 5.54%, 18.06%, 4997501), (MIDDLE EAST, 2019, 69.90%, 3.98%, 5.96%, 20.16%, 335843), (MIDDLE EAST, 2023, 73.07%, 3.11%, 5.01%, 18.82%, 219680), (NORTH AMERICA, 2019, 70.18%, 4.15%, 5.19%, 20.49%, 2356946), (NORTH AMERICA, 2023, 73.40%, 2.26%, 5.11%, 19.22%, 2142430), (SOUTH AMERICA INC CARIBBEAN, 2019, 76.53%, 2.45%, 3.59%, 17.43%, 200103), (SOUTH AMERICA INC CARIBBEAN, 2023, 78.17%, 1.51%, 3.06%, 17.26%, 196972), (UK, 2019, 75.78%, 0.42%, 5.68%, 18.13%, 1151701), (UK, 2023, 77.84%, 0.36%, 5.00%, 16.80%, 1163232)]

### By aircraft

In [96]:
df_pivot_groupby_acft = group_pivot(pivot_df, ['IATA_AC_TYP_CD','ACT_AC_TYP_CD','FIRST_SEATS_QTY','CLUB_SEATS_QTY','PREM_ECONOMY_SEATS_QTY','ECONOMY_SEATS_QTY'])
group_pivot_to_lookup(df_pivot_groupby_acft)

Empty DataFrame
Columns: []
Index: [(319, A4, 0.0, 16.0, 0.0, 119.0, 74.93%, 0.52%, 6.65%, 17.90%, 2543628), (319, M4, 0.0, 21.0, 0.0, 113.0, 79.05%, 0.28%, 4.86%, 15.81%, 434277), (320, A3, 0.0, 24.0, 0.0, 132.0, 76.78%, 0.41%, 5.91%, 16.90%, 1991188), (320, E9, 0.0, 29.0, 0.0, 130.0, 75.33%, 0.30%, 5.01%, 19.37%, 198995), (320, M2, 0.0, 24.0, 0.0, 144.0, 80.36%, 0.27%, 4.37%, 15.00%, 50391), (320, R3, 0.0, 28.0, 0.0, 139.0, 76.25%, 0.37%, 5.57%, 17.80%, 2359785), (320, T3, 0.0, 26.0, 0.0, 142.0, 79.33%, 0.27%, 4.34%, 16.06%, 168421), (321, M6, 0.0, 23.0, 0.0, 131.0, 79.23%, 0.59%, 5.74%, 14.44%, 136777), (321, V6, 0.0, 38.0, 0.0, 160.0, 76.70%, 0.43%, 5.95%, 16.92%, 1013806), (321, W6, 0.0, 42.0, 0.0, 154.0, 77.34%, 0.44%, 5.67%, 16.56%, 204879), (32A, H3, 0.0, 20.0, 0.0, 150.0, 75.87%, 0.42%, 5.89%, 17.82%, 1086458), (32N, N3, 0.0, 20.0, 0.0, 150.0, 75.78%, 0.39%, 5.67%, 18.16%, 1701447), (32Q, N6, 0.0, 32.0, 0.0, 172.0, 76.59%, 0.36%, 5.15%, 17.90%, 889836), (332, W9, 0.0, 30.0, 21.0, 235.0, 84.89%, 0.38%, 4.97%, 9.75%, 523), (339, W9, 0.0, 30.0, 21.0, 234.0, 84.56%, 0.18%, 3.68%, 11.57%, 23096), (343, W9, 0.0, 45.0, 0.0, 212.0, 80.09%, 0.33%, 3.87%, 15.72%, 43925), (351, K5, 0.0, 56.0, 56.0, 219.0, 79.38%, 0.26%, 4.10%, 16.26%, 582984), (388, A8, 14.0, 97.0, 55.0, 303.0, 72.40%, 3.20%, 5.08%, 19.33%, 1243139), (744, S4, 14.0, 86.0, 30.0, 145.0, 60.85%, 5.19%, 7.03%, 26.92%, 675347), (744, V4, 14.0, 52.0, 36.0, 235.0, 76.80%, 3.79%, 3.83%, 15.58%, 715726), (777, D7, 0.0, 48.0, 40.0, 184.0, 78.42%, 0.25%, 3.97%, 17.37%, 402338), (777, J7, 0.0, 32.0, 48.0, 252.0, 74.31%, 0.00%, 5.56%, 20.14%, 144), (777, K7, 0.0, 48.0, 24.0, 203.0, 78.21%, 0.00%, 3.85%, 17.95%, 156), (777, L7, 0.0, 48.0, 24.0, 203.0, 78.30%, 0.27%, 4.38%, 17.05%, 155095), (777, M7, 7.0, 50.0, 40.0, 138.0, 71.40%, 3.33%, 5.93%, 19.34%, 450916), (777, P7, 17.0, 48.0, 24.0, 127.0, 65.77%, 6.51%, 5.97%, 21.75%, 108147), (777, R7, 10.0, 52.0, 40.0, 134.0, 69.67%, 3.98%, 6.81%, 19.54%, 45019), (777, V7, 14.0, 48.0, 40.0, 124.0, 68.16%, 5.21%, 5.92%, 20.71%, 538568), (777, W7, 12.0, 48.0, 32.0, 127.0, 68.59%, 5.15%, 5.11%, 21.16%, 270777), (77W, F7, 8.0, 76.0, 40.0, 130.0, 63.94%, 3.36%, 6.84%, 25.86%, 374665), (77W, G7, 14.0, 56.0, 44.0, 183.0, 71.83%, 4.40%, 4.99%, 18.77%, 525688), (781, X8, 7.0, 49.0, 35.0, 165.0, 74.74%, 2.78%, 4.31%, 18.16%, 243361), (788, B8, 0.0, 35.0, 25.0, 154.0, 80.93%, 0.19%, 3.22%, 15.66%, 647096), (789, L8, 8.0, 42.0, 39.0, 127.0, 73.59%, 2.65%, 4.64%, 19.11%, 835862)]

In [111]:
pivot_df.head(5)

Lounge_eligibility_tier,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TS,OPG_FLT_NO,ACT_DEP_TML_CD,PLND_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passesgers
0,2019-03-31103,103,2019-03-31,2019-03-31 17:30:00,103.0,5,YYC,788,B8,2019-03-31,17:30:00,CA,Canada,CANADA,NORTH AMERICA,WB,0.0,35.0,25.0,154.0,2019,154,1,9,33,197
1,2019-03-31105,105,2019-03-31,2019-03-31 21:25:00,105.0,5,DXB,777,V7,2019-03-31,21:25:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,137,13,13,46,209
2,2019-03-31107,107,2019-03-31,2019-03-31 11:50:00,107.0,5,DXB,777,V7,2019-03-31,11:50:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,126,11,18,46,201
3,2019-03-31109,109,2019-03-31,2019-03-31 20:15:00,109.0,5,DXB,777,V7,2019-03-31,20:15:00,AE,United Arab Emirates,SOUTHERN GULF,MIDDLE EAST,WB,14.0,48.0,40.0,124.0,2019,139,14,22,42,217
4,2019-03-3111,11,2019-03-31,2019-03-31 18:55:00,11.0,5,SIN,388,A8,2019-03-31,18:55:00,SG,Singapore,FAR EAST JSA HUBS,AUSTRALASIA PACIFIC,WB,14.0,97.0,55.0,303.0,2019,308,17,43,75,443


In [190]:
def string_to_cat(df):
    fields = df.columns
    for f in fields:
        if type(df[f][0]) == str:
            df[f], _ = pd.factorize(df[f])
    return df

pivot_df_cat = string_to_cat(pivot_df)

In [191]:
print(pivot_df_cat.dtypes)

Lounge_eligibility_tier
Skew_Id                            int64
OPERATING_FLT_NO                   int64
GMT_UPLIFT_DT                     object
GMT_PLND_DEP_TS           datetime64[ns]
OPG_FLT_NO                       float64
ACT_DEP_TML_CD                     int64
PLND_ARR_STN_CD                    int64
IATA_AC_TYP_CD                     int64
ACT_AC_TYP_CD                      int64
GMT_PLND_DEP_DT                   object
GMT_PLND_DEP_TIME                 object
COUNTRY_CD                         int64
COUNTRY_NM                         int64
CORP_GEOG_CTRY_GRP_NM              int64
CORP_GEOG_CONTINENT_NM             int64
WB_NB_CAT                          int64
FIRST_SEATS_QTY                  float64
CLUB_SEATS_QTY                   float64
PREM_ECONOMY_SEATS_QTY           float64
ECONOMY_SEATS_QTY                float64
GMT_UPLIFT_YR                      int64
Not eligible                       int64
Tier 1                             int64
Tier 2                           

In [192]:
pivot_df_cat.head(3)

Lounge_eligibility_tier,Skew_Id,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TS,OPG_FLT_NO,ACT_DEP_TML_CD,PLND_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passesgers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio
0,0,103,2019-03-31,2019-03-31 17:30:00,103.0,0,0,0,0,2019-03-31,17:30:00,0,0,0,0,0,0.0,35.0,25.0,154.0,2019,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513
1,1,105,2019-03-31,2019-03-31 21:25:00,105.0,0,1,1,1,2019-03-31,21:25:00,1,1,1,1,0,14.0,48.0,40.0,124.0,2019,137,13,13,46,209,0.655502,0.062201,0.062201,0.220096
2,2,107,2019-03-31,2019-03-31 11:50:00,107.0,0,1,1,1,2019-03-31,11:50:00,1,1,1,1,0,14.0,48.0,40.0,124.0,2019,126,11,18,46,201,0.626866,0.054726,0.089552,0.228856


In [193]:
pivot_df_no_skew = pivot_df.iloc[:,1:]

In [194]:
print(pivot_df_no_skew.shape)
pivot_df_no_skew.head(5)

(130976, 29)


Lounge_eligibility_tier,OPERATING_FLT_NO,GMT_UPLIFT_DT,GMT_PLND_DEP_TS,OPG_FLT_NO,ACT_DEP_TML_CD,PLND_ARR_STN_CD,IATA_AC_TYP_CD,ACT_AC_TYP_CD,GMT_PLND_DEP_DT,GMT_PLND_DEP_TIME,COUNTRY_CD,COUNTRY_NM,CORP_GEOG_CTRY_GRP_NM,CORP_GEOG_CONTINENT_NM,WB_NB_CAT,FIRST_SEATS_QTY,CLUB_SEATS_QTY,PREM_ECONOMY_SEATS_QTY,ECONOMY_SEATS_QTY,GMT_UPLIFT_YR,Not eligible,Tier 1,Tier 2,Tier 3,Total_Passesgers,Not eligible ratio,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio
0,103,2019-03-31,2019-03-31 17:30:00,103.0,0,0,0,0,2019-03-31,17:30:00,0,0,0,0,0,0.0,35.0,25.0,154.0,2019,154,1,9,33,197,0.781726,0.005076,0.045685,0.167513
1,105,2019-03-31,2019-03-31 21:25:00,105.0,0,1,1,1,2019-03-31,21:25:00,1,1,1,1,0,14.0,48.0,40.0,124.0,2019,137,13,13,46,209,0.655502,0.062201,0.062201,0.220096
2,107,2019-03-31,2019-03-31 11:50:00,107.0,0,1,1,1,2019-03-31,11:50:00,1,1,1,1,0,14.0,48.0,40.0,124.0,2019,126,11,18,46,201,0.626866,0.054726,0.089552,0.228856
3,109,2019-03-31,2019-03-31 20:15:00,109.0,0,1,1,1,2019-03-31,20:15:00,1,1,1,1,0,14.0,48.0,40.0,124.0,2019,139,14,22,42,217,0.640553,0.064516,0.101382,0.193548
4,11,2019-03-31,2019-03-31 18:55:00,11.0,0,2,2,2,2019-03-31,18:55:00,2,2,2,2,0,14.0,97.0,55.0,303.0,2019,308,17,43,75,443,0.695260,0.038375,0.097065,0.169300


In [196]:
# pivot_df_no_skew.apply(lambda x: pd.factorize(x)[0]).corr()[['Tier 1 ratio', 'Tier 2 ratio', 'Tier 3 ratio']]
pivot_df_no_skew.corr()[['Tier 1 ratio', 'Tier 2 ratio', 'Tier 3 ratio']]

Lounge_eligibility_tier,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio
Lounge_eligibility_tier,,,
OPERATING_FLT_NO,-0.436034,0.000373,-0.148677
OPG_FLT_NO,-0.436034,0.000373,-0.148677
ACT_DEP_TML_CD,-0.076620,-0.083304,-0.126168
PLND_ARR_STN_CD,-0.284155,-0.021552,-0.100469
IATA_AC_TYP_CD,-0.497705,0.022559,-0.096089
ACT_AC_TYP_CD,-0.235550,-0.049697,-0.044133
COUNTRY_CD,-0.358135,0.030172,-0.184622
COUNTRY_NM,-0.358135,0.030172,-0.184622
CORP_GEOG_CTRY_GRP_NM,-0.326903,0.139268,-0.129287


In [189]:
pivot_df_no_skew.apply(lambda x: pd.factorize(x)[0]).dtypes

Lounge_eligibility_tier
OPERATING_FLT_NO          int64
GMT_UPLIFT_DT             int64
GMT_PLND_DEP_TS           int64
OPG_FLT_NO                int64
ACT_DEP_TML_CD            int64
PLND_ARR_STN_CD           int64
IATA_AC_TYP_CD            int64
ACT_AC_TYP_CD             int64
GMT_PLND_DEP_DT           int64
GMT_PLND_DEP_TIME         int64
COUNTRY_CD                int64
COUNTRY_NM                int64
CORP_GEOG_CTRY_GRP_NM     int64
CORP_GEOG_CONTINENT_NM    int64
WB_NB_CAT                 int64
FIRST_SEATS_QTY           int64
CLUB_SEATS_QTY            int64
PREM_ECONOMY_SEATS_QTY    int64
ECONOMY_SEATS_QTY         int64
GMT_UPLIFT_YR             int64
Not eligible              int64
Tier 1                    int64
Tier 2                    int64
Tier 3                    int64
Total_Passesgers          int64
Not eligible ratio        int64
Tier 1 ratio              int64
Tier 2 ratio              int64
Tier 3 ratio              int64
dtype: object

In [184]:
correlation_table = pivot_df_cat.corr()

correlation_table[['Tier 1 ratio', 'Tier 2 ratio', 'Tier 3 ratio']]

Lounge_eligibility_tier,Tier 1 ratio,Tier 2 ratio,Tier 3 ratio
Lounge_eligibility_tier,,,
OPERATING_FLT_NO,-0.436034,0.000373,-0.148677
OPG_FLT_NO,-0.436034,0.000373,-0.148677
FIRST_SEATS_QTY,0.805474,-0.029791,0.172066
CLUB_SEATS_QTY,0.603017,-0.061284,0.162735
PREM_ECONOMY_SEATS_QTY,0.618617,-0.099131,0.112705
ECONOMY_SEATS_QTY,0.186736,-0.110548,-0.013479
GMT_UPLIFT_YR,-0.174400,-0.106071,0.005113
Not eligible,0.272826,-0.300299,-0.196056
Tier 1,0.903737,0.057316,0.186073


AttributeError: 'DataFrame' object has no attribute 'Sentence'

<br><br><br>

In [97]:
#--------------------------------
#
# Your turn!!!
#
#--------------------------------

<u>What level of granularity do you use?</u>
- ... (your answer here)

<u>What metric do you use to come up with Lounge eligibility profiles?</u>
- ... (your answer here)

In [98]:
# your code here

<br>

Please save your final lookup table below in the form of a pandas dataframe. It must contain the categories you have come up with as rows, and the Tier 1, Tier 2, and Tier 3 percentage of costumers as columns.